# Applications of SOS Programming in Flowpipe Construction

Flowpipe construction consists of under or over-approximating the sets of states reachable by dynamical systems. Recently a method has been developed for the class of polynomial ODEs with uncertain initial states (see [1], abbreviated `XFZ18under`). This method consists of reducing the Hamilton-Jacobi-Bellman equation to a hierarchy of semidefinite programs.

In this notebook we consider the problem of approximating the flowpipe of a system of polynomial ODEs using `XFZ18under`. This is a Julia implementation that relies on the JuMP ecosystem (`JuMP`, `PolyJuMP`, `SumOfSquares`, `MathProgInterface`, `MathOptInterfaceMosek`) and the `JuliaAlgebra` ecosystem (`MultivariatePolynomials`, `DynamicPolynomials`). The implementation is evaluated on a set of standard benchmarks from formal verification and control engineering domains.

---

**References:**

- [1] Xue, B., Fränzle, M., & Zhan, N. (2018, April). [Under-Approximating Reach Sets for Polynomial Continuous Systems. In Proceedings of the 21st International Conference on Hybrid Systems: Computation and Control (part of CPS Week) (pp. 51-60). ACM.](https://dl.acm.org/citation.cfm?id=3178133)

**Quick links to documentation:**

- www.juliaopt.org/SumOfSquares.jl/latest/
- https://sums-of-squares.github.io/sos/index.html

## Van-der-Pol system

Consider the following van-der-Pol system:

$$
\dot{x}_1 = x_2 \\
\dot{x}_2 = -0.2x_1 + x_2 - 0.2x_1^2 x_2
$$

In [1]:
using MultivariatePolynomials,
      JuMP,
      PolyJuMP,
      SumOfSquares,
      DynamicPolynomials,
      MathOptInterfaceMosek,
      MathematicalSystems,
      MosekTools
 
const ∂ = differentiate

differentiate (generic function with 18 methods)

In [2]:
# symbolic variables
@polyvar x₁ x₂ t

# time duration (scaled, see dynamics below)
T = 1.0 

# dynamics
f = 2 * [x₂, -0.2*x₁ + x₂ - 0.2*x₁^2*x₂] 

# set of initial states X₀ = {x: V₀(x) <= 0}
V₀ = x₁^2 + x₂^2 - 0.25

# constraints Y = {x: g(x) >= 0} compact search space Y x [0, T]
g = 25 - x₁^2 - x₂^2

# degree of the relaxation
k = 12

# monomial vector up to order k, 0 <= sum_i alpha_i <= k, if alpha_i is the exponent of x_i
X = monomials([x₁, x₂], 0:k)
XT = monomials([x₁, x₂, t], 0:k)

# create a SOS JuMP model to solve with Mosek
model = SOSModel(with_optimizer(Mosek.Optimizer))

# add unknown Φ to the model
@variable(model, Φ, Poly(XT))

# jacobian
∂t = α -> ∂(α, t)
∂xf = α -> ∂(α, x₁) * f[1] + ∂(α, x₂) * f[2] 
LΦ = ∂t(Φ) + ∂xf(Φ)

# Φ(x, t) at time 0
Φ₀ = subs(Φ, t => 0.)

# scalar variable
@variable(model, ϵ)

dom1 = @set t*(T-t) >= 0 && g >= 0
dom2 = @set g >= 0
@constraint(model, ϵ >= 0.)
@constraint(model, LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, ϵ - LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, Φ₀ - V₀ ∈ SOSCone(), domain = dom2)
@constraint(model, ϵ + V₀ - Φ₀ ∈ SOSCone(), domain = dom2)

@objective(model, Min, ϵ)

ϵ

In [3]:
optimize!(model)

println("Relaxation order : k = $k")
println("JuMP.termination_status(model) = ", JuMP.termination_status(model))
println("JuMP.primal_status(model) = ", JuMP.primal_status(model))
println("JuMP.dual_status(model) = ", JuMP.dual_status(model))
println("JuMP.objective_bound(model) = ", JuMP.objective_bound(model))
println("JuMP.objective_value(model) = ", JuMP.objective_value(model))

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1543            
  Cones                  : 0               
  Scalar variables       : 456             
  Matrix variables       : 10              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimizat

### Comparison with a YALMIP implementation in MATLAB

|  Package    | k    |Constraints|Scalar variables|Matrix variables|Time(s)|
|-------------|------|-----------|----------------|----------------|-------|
|JuMP v0.18.5         |    2 |       245 |            175 |              8 |   < 1 |
|**JuMP v0.19.0**         |    2 |    83    |      13       |            8   |   < 1 |
|YALMIP       |    2 |       152 |             63 |              10|   < 1 |
||
|JuMP v0.18.5         |    3 |      893 |           715 |              10|   ~ 1 |
|**JuMP v0.19.0**         |    3 |    199    |        21     |        10       |   < 1 |
|YALMIP       |    3 |       254 |            121 |              10|   ~ 1 |
||
|JuMP v0.18.5         |    4 |      893 |           730 |              10|  ~1 |
|**JuMP v0.19.0**         |    4 |   199     |       36      |      10         |   < 1 |
|YALMIP       |    4 |       394 |           206  |              10|  1.18 |
||
|JuMP v0.18.5         |    5 |       2639 |            2309 |              10 |   1.17 |
|**JuMP v0.19.0**         |    5 |     387   |       57      |         10      |   < 1 |
|YALMIP       |    5 |      578  |         323     |          10    |  0.11 |
||
|JuMP v0.18.5         |    6 |       2639 |         2337   |              10|   0.90 |
|**JuMP v0.19.0**         |    6 |    387    |       85      |       10        |   < 1 |
|YALMIP       |    6 |    812    |         477    |    10          |  1.10  |
||
|JuMP v0.18.5         |    7 |      6725 |       6183     |              10|  5.62 |
|**JuMP v0.19.0**         |    7 |   663     |     121        |      10         |   < 1 |
|YALMIP       |    7 |     1102   |        673     |        10      |  1.52 |
||
|JuMP v0.18.5         |    8 |     6725  |      6228       |          10    |   6.06 |
|**JuMP v0.19.0**         |    8 |  663      |        166     |           10    |   < 1 |
|YALMIP       |    8 |    1454    |       916       |          10    |  1.10 |
||
|JuMP v0.18.5         |    9 |    15269   |      14447      |           10   |  41.2  |
|**JuMP v0.19.0**         |    9 |      1043  |        221     |        10       |   1.70 |
|YALMIP       |    9 |   1874     |       1211     |         10     |  1.58  |
||
|JuMP v0.18.5         |   10 |    15269   |     14513       |      10        |  38.1 |
|**JuMP v0.19.0**         |    10 |    1043    |    287         |     10          |   1.67 |
|YALMIP       |   10 |     2368   |       1563     |        10      |  2.73 |
||
|JuMP v0.18.5         |   11 |   31617    |     30439       |     10         | 244  |
|**JuMP v0.19.0**         |    11 |   1543     |      365       |     10          |   4.88 |
|YALMIP       |   11 |   2942    |       1977      |       10       |  2.30 |
||
|JuMP v0.18.5         |   12 |     31617  |     30530       |       10       | 231 |
|**JuMP v0.19.0**         |    12 | 1543       |   456          |         10      |  5.02 |
|YALMIP       |   12 |    3602    |      2458       |      10        | 6.57  |

Related: https://github.com/JuliaOpt/MosekTools.jl/issues/11.

In [4]:
MOI.get(model, MOI.SolveTime())

STALL


5.254102945327759

## Extracting the under and over approximations

In [5]:
# Recovering the solution:
ϵopt = JuMP.objective_value(model)

# Punder <= 0
Punder = subs(JuMP.value(model[:Φ]), t => T)

STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STAL

-3.254406926195832e-6x₁¹² + 3.834217164793429e-6x₁¹¹x₂ - 6.3384127954307375e-6x₁¹⁰x₂² + 1.7821158886139766e-6x₁⁹x₂³ - 3.874093809241813e-7x₁⁸x₂⁴ + 1.5483592082483843e-6x₁⁷x₂⁵ - 1.7690055890869638e-6x₁⁶x₂⁶ - 9.798790301107213e-7x₁⁵x₂⁷ + 5.210797975903119e-7x₁⁴x₂⁸ + 1.3500407479164792e-6x₁³x₂⁹ + 4.4924259835334384e-7x₁²x₂¹⁰ - 7.166251275333759e-8x₁x₂¹¹ - 1.5032013847766846e-7x₂¹² + 1.4831621204102143e-20x₁¹¹ + 1.3691290786000017e-20x₁¹⁰x₂ - 5.0093500950317737e-20x₁⁹x₂² + 4.3677787535910767e-20x₁⁸x₂³ - 3.173767662076143e-20x₁⁷x₂⁴ + 1.9724120886883362e-20x₁⁶x₂⁵ + 3.932019772930437e-20x₁⁵x₂⁶ - 3.366118784605938e-20x₁⁴x₂⁷ + 7.335098065117538e-21x₁³x₂⁸ + 1.8928666158525362e-21x₁²x₂⁹ - 7.972721076141193e-21x₁x₂¹⁰ + 3.58418280872306e-21x₂¹¹ + 0.00015991576845668712x₁¹⁰ - 0.0004657855037397151x₁⁹x₂ + 0.000589085715209256x₁⁸x₂² - 0.0002325700371386792x₁⁷x₂³ - 3.930451038774872e-5x₁⁶x₂⁴ + 3.2322704287292744e-6x₁⁵x₂⁵ + 1.3335992567483893e-6x₁⁴x₂⁶ + 2.7518735707131795e-5x₁³x₂⁷ - 1.323325367954451e-5

In [6]:
# Pover <= 0
Pover = subs(JuMP.value(model[:Φ]), t => T) - ϵopt * (T+1)

STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STALL
STAL

-3.254406926195832e-6x₁¹² + 3.834217164793429e-6x₁¹¹x₂ - 6.3384127954307375e-6x₁¹⁰x₂² + 1.7821158886139766e-6x₁⁹x₂³ - 3.874093809241813e-7x₁⁸x₂⁴ + 1.5483592082483843e-6x₁⁷x₂⁵ - 1.7690055890869638e-6x₁⁶x₂⁶ - 9.798790301107213e-7x₁⁵x₂⁷ + 5.210797975903119e-7x₁⁴x₂⁸ + 1.3500407479164792e-6x₁³x₂⁹ + 4.4924259835334384e-7x₁²x₂¹⁰ - 7.166251275333759e-8x₁x₂¹¹ - 1.5032013847766846e-7x₂¹² + 1.4831621204102143e-20x₁¹¹ + 1.3691290786000017e-20x₁¹⁰x₂ - 5.0093500950317737e-20x₁⁹x₂² + 4.3677787535910767e-20x₁⁸x₂³ - 3.173767662076143e-20x₁⁷x₂⁴ + 1.9724120886883362e-20x₁⁶x₂⁵ + 3.932019772930437e-20x₁⁵x₂⁶ - 3.366118784605938e-20x₁⁴x₂⁷ + 7.335098065117538e-21x₁³x₂⁸ + 1.8928666158525362e-21x₁²x₂⁹ - 7.972721076141193e-21x₁x₂¹⁰ + 3.58418280872306e-21x₂¹¹ + 0.00015991576845668712x₁¹⁰ - 0.0004657855037397151x₁⁹x₂ + 0.000589085715209256x₁⁸x₂² - 0.0002325700371386792x₁⁷x₂³ - 3.930451038774872e-5x₁⁶x₂⁴ + 3.2322704287292744e-6x₁⁵x₂⁵ + 1.3335992567483893e-6x₁⁴x₂⁶ + 2.7518735707131795e-5x₁³x₂⁷ - 1.323325367954451e-5

### Visualization using IntervalConstraintProgramming

See notebook `Constraints.ipynb`.

In [ ]:
# no necesito convertir, si utilizo el IntervalContractors#fix_zero_power
using ModelingToolkit

# TODO: temporary definition here, needs convert(Variable, z^2 + t^2)
vars = ModelingToolkit.@variables x1 x2

function tomtexpr(poly::Polynomial)
    global vars
    expr = 0.0
    for (i, term) in enumerate(poly.x)
        if iszero(exponents(term))
            expr += poly.a[i]
        else
            expr += poly.a[i] * prod([vars[j]^value for (j, value) in enumerate(exponents(term)) if value != 0])
        end
    end
    return expr
end

In [ ]:
Punder_mt = tomtexpr(Punder)

In [ ]:
include("constraints.jl")

using Plots

In [ ]:
S = Separator(-Inf..0.0, Punder_mt)

In [ ]:
p = pave(S, IntervalBox(-2..2, 2), 0.1)

In [ ]:
S = Separator(-Inf..0.0, Punder_mt)
p = pave(S, IntervalBox(-3..3, 2), 0.1)

plot(p.inner, lw=0, aspectratio=1, label="inner")
plot!(p.boundary, lw=0, label="boundary", legend=:topleft)

In [ ]:
Punder(2.0, 0.5)

In [ ]:
S = Separator(-Inf..0.0, Punder_mt)
p = pave(S, IntervalBox(-2..2, 2), 0.1)

Pover_mt = tomtexpr(Pover)
Sover = Separator(-Inf..0.0, Pover_mt)
pover = pave(Sover, IntervalBox(-2..2, 2), 0.1)

plot(p.inner, lw=0, aspectratio=1, label="under", alpha=.4)
plot!(pover.inner, lw=0, label="over", legend=:topleft, alpha=.4)

## Visualization

In [ ]:
using LinearAlgebra, StaticPolynomials

# filter out the smallest coefficients?
filter_coeffs = false

if filter_coeffs
    max_coeff = maximum(abs.(Punder.a))

    # plots are *very* sensitive to small values
    # make tol bigger than 0 to filter out some coefficients
    tol = 0.0 # max_coeff / 1e19
    kept_coeffs = map(c -> abs(c) > tol, Punder.a)
    Punder = dot(Punder.a[kept_coeffs], Punder.x[kept_coeffs])
    Pover = dot(Pover.a[kept_coeffs], Pover.x[kept_coeffs]);
end

# we now convert to a static polynomial for faster evaluation
Punder_st = StaticPolynomials.Polynomial(Punder)
Pover_st = StaticPolynomials.Polynomial(Pover);

_Punder(x, y) = Punder_st([x, y])
_Pover(x, y) = Pover_st([x, y])

In [ ]:
@btime Punder(1.0, 1.0)

In [ ]:
@btime Punder_st([1.0, 1.0])

In [ ]:
2e-6 / 96e-9

In [ ]:
Punder_st

### Plots using ImplicitEquations

In [ ]:
using ImplicitEquations, Plots
gr()

In [ ]:
G = plot()
plot!(G, _Punder ⩵ 0., xlims=(-4, 4), ylims=(-4, 4), color="red")
plot!(G, _Pover ⩵ 0., xlims=(-4, 4), ylims=(-4, 4), color="blue")
G

In [ ]:
G = plot()
Gu = plot(_Punder ≪ 0., xlims=(-2, 2), ylims=(-2, 2))
Go = plot(_Pover ≪ 0., xlims=(-2, 2), ylims=(-2, 2))
plot(Gu, Go)

In [ ]:
Punder(2.0, 1.5)

### Plots using IntervalConstraintProgramming

In [ ]:
using IntervalConstraintProgramming, ValidatedNumerics

In [ ]:
@function Funder(x₁, x₂) = _Punder(x₁, x₂)

In [ ]:
Bx₁x₂ = IntervalBox(-10..10, -10..10)

In [ ]:
Cunder = IntervalConstraintProgramming.@constraint Funder(x₁, x₂) <= 0.0 # no

In [ ]:
paving = pave(Cunder, Bx₁x₂)

## Adding a constraint

In [17]:
# symbolic variables
@polyvar x₁ x₂ t

# time duration (scaled, see dynamics below)
T = 1.0 

# dynamics
f = 2 * [x₂, -0.2*x₁ + x₂ - 0.2*x₁^2*x₂] 

# set of initial states X₀ = {x: V₀(x) <= 0}
V₀ = x₁^2 + x₂^2 - 0.25

# constraints Y = {x: g(x) >= 0} compact search space Y x [0, T]
g = 25 - x₁^2 - x₂^2

# degree of the relaxation
k = 12

# monomial vector up to order k, 0 <= sum_i alpha_i <= k, if alpha_i is the exponent of x_i
X = monomials([x₁, x₂], 0:k)
XT = monomials([x₁, x₂, t], 0:k)

# create a SOS JuMP model to solve with Mosek
model = SOSModel(with_optimizer(Mosek.Optimizer))

# add unknown Φ to the model
@variable(model, Φ, Poly(XT))

# jacobian
∂t = α -> ∂(α, t)
∂xf = α -> ∂(α, x₁) * f[1] + ∂(α, x₂) * f[2] 
LΦ = ∂t(Φ) + ∂xf(Φ)

# Φ(x, t) at time 0
Φ₀ = subs(Φ, t => 0.)

# scalar variable
@variable(model, ϵ)

dom1 = @set t*(T-t) >= 0 && g >= 0
dom2 = @set g >= 0
@constraint(model, ϵ >= 0.)
@constraint(model, LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, ϵ - LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, Φ₀ - V₀ ∈ SOSCone(), domain = dom2)
@constraint(model, ϵ + V₀ - Φ₀ ∈ SOSCone(), domain = dom2)

# add a safety constarint
#@constraint(model, x₂ <= 100.)

@objective(model, Min, ϵ)

ϵ

In [18]:
optimize!(model)

println("Relaxation order : k = $k")
println("JuMP.termination_status(model) = ", JuMP.termination_status(model))
println("JuMP.primal_status(model) = ", JuMP.primal_status(model))
println("JuMP.dual_status(model) = ", JuMP.dual_status(model))
println("JuMP.objective_bound(model) = ", JuMP.objective_bound(model))
println("JuMP.objective_value(model) = ", JuMP.objective_value(model))

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1543            
  Cones                  : 0               
  Scalar variables       : 456             
  Matrix variables       : 10              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimizat

In [ ]:
# symbolic variables
@polyvar x₁ x₂ t

# time duration (scaled, see dynamics below)
T = 1.0 

# dynamics
f = 2 * [x₂, -0.2*x₁ + x₂ - 0.2*x₁^2*x₂] 

# set of initial states X₀ = {x: V₀(x) <= 0}
V₀ = x₁^2 + x₂^2 - 0.25

# constraints Y = {x: g(x) >= 0} compact search space Y x [0, T]
g = 25 - x₁^2 - x₂^2

# degree of the relaxation
k = 12

# monomial vector up to order k, 0 <= sum_i alpha_i <= k, if alpha_i is the exponent of x_i
X = monomials([x₁, x₂], 0:k)
XT = monomials([x₁, x₂, t], 0:k)

# create a SOS JuMP model to solve with Mosek
model_verif = SOSModel(with_optimizer(Mosek.Optimizer))

# add unknown Φ to the model
@variable(model, Φ, Poly(XT))

# jacobian
∂t = α -> ∂(α, t)
∂xf = α -> ∂(α, x₁) * f[1] + ∂(α, x₂) * f[2] 
LΦ = ∂t(Φ) + ∂xf(Φ)

# Φ(x, t) at time 0
Φ₀ = subs(Φ, t => 0.)

# scalar variable
@variable(model, ϵ)

dom1 = @set t*(T-t) >= 0 && g >= 0
dom2 = @set g >= 0
@constraint(model, ϵ >= 0.)
@constraint(model, LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, ϵ - LΦ ∈ SOSCone(), domain = dom1)
@constraint(model, Φ₀ - V₀ ∈ SOSCone(), domain = dom2)
@constraint(model, ϵ + V₀ - Φ₀ ∈ SOSCone(), domain = dom2)

# add a safety constarint
#@constraint(model, x₂ <= 100.)

@objective(model, Min, ϵ)

In [ ]:
Punder

In [29]:
model_verif = Model(with_optimizer(Mosek.Optimizer))

@variable(model_verif, t)
@variable(model_verif, x₁)
@variable(model_verif, x₂)

@constraint(model_verif, Punder <= 0)

@objective(model_verif, Max, x₂)

optimize!(model_verif)

ErrorException: PolyJuMP is just a JuMP extension for modelling Polynomial Optimization: it does not implement any reformulation. To use automatic sums of squares (SOS) reformulations, install the SumOfSquares Julia package and try `using SumOfSquares` and `setpolymodule!(SumOfSquares)` or use `SOSModel` instead of `Model`.